In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

import os 
import numpy as np

### Parameters

In [2]:
epochs = 200
batch_size = 128
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8 
latent_dim = 100
n_classes = 10 
img_size = 28
channels = 1 
sample_interval = 400
k = 1
img_shape = (channels, img_size, img_size)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
#     transforms.Resize(img_size, img_size),
])

# to_pil_image = transforms.ToPILImage() # This is required when we want to save the generated images as a .gif file.

train_data = datasets.MNIST(
    root='./input',
    train=True,
    download=True,
    transform=transform,
)

train_loader = DataLoader(
    train_data, 
    batch_size=batch_size, 
    shuffle=True
)

In [4]:
sample_batch = next(iter(train_loader))
sample_batch[0][0].shape

torch.Size([1, 28, 28])

### Utils Functions

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def make_dir(path='./output/image'):
    if not os.path.exists(path):
        os.makedirs(path)
make_dir()
        
def latent_vector(size, latent_dim):
    return torch.randn(size, latent_dim).to(device)

def save_generated_image(image, path):
    save_image(image, path)

def label_real(size):
    data = torch.ones(size, 1)
    return data.to(device)

def label_fake(size):
    data = torch.ones(size, 1)
    return data.to(device)

### Generator Network

In [6]:
class GenNet(nn.Module):
    def __init__(self):
        super(GenNet, self).__init__()
        
        self.label_emb = nn.Embedding(n_classes, n_classes)
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            
            return layers
        
        self.model = nn.Sequential(
            *block(latent_dim+n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
    
    def forward(self, noise, labels):
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        
        return img

### Discriminator Network

In [7]:
class DisNet(nn.Module):
    def __init__(self):
        super(DisNet, self).__init__()
        
        self.label_embedding = nn.Embedding(n_classes, n_classes)
        
        def block(in_feat, out_feat, is_dropout=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if is_dropout:
                layers.append(nn.Dropout(0.4))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
            
        self.model = nn.Sequential(
            *block(n_classes + int(np.prod(img_shape)), 512, is_dropout=False),
            *block(512, 512),
            *block(512, 512),
            nn.Linear(512, 1),
        )
    
    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

In [8]:
generator = GenNet().to(device)
discriminator = DisNet().to(device)

In [9]:
print(generator)
print(discriminator)

GenNet(
  (label_emb): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=784, bias=True)
    (12): Tanh()
  )
)
DisNet(
  (label_embedding): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=794, out_features=512, b

### Optimizer

In [10]:
opt_D = optim.Adam(discriminator.parameters(), lr=lr)
opt_G = optim.Adam(generator.parameters(), lr=lr)

In [11]:
print(opt_D)
print(opt_G)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)


### Loss Function

In [12]:
criterion = nn.MSELoss()

### Discriminator Training

In [13]:
# def train_discriminator(optim, real_data, fake_data, label):
#     b_size = real_data.size(0)
    
#     real_label = label_real(b_size)
#     fake_label = label_fake(b_size)
    
#     optim.zero_grad()
    
#     output_real = discriminator(real_data, label)
#     loss_real = criterion(output_real, real_label)
    
#     output_fake = discriminator(fake_data, label)
#     loss_fake = criterion(output_fake, fake_label)
    
#     loss = (loss_real+loss_fake)/2
#     loss.backward()
#     optim.step()

#     return loss

### Generator Training

In [14]:
# def train_generator(optim, fake_data, label):
    
#     b_size = fake_data.size(0)
#     real_label = label_real(b_size)
    
#     optim.zero_grad()
#     output = discriminator(fake_data, label)
#     loss = criterion(output, real_label)
    
#     loss.backward()
#     optim.step()
#     return loss


### Training Loop

In [15]:
# losses_g = []
# losses_d = []
# images = []

# for epoch in range(epochs):
#     loss_g = 0.0 
#     loss_d = 0.0 
#     for i, (image, label) in enumerate(train_loader):
#         image = image.to(device)
#         b_size = image.size(0)
#         noise = latent_vector(b_size, latent_dim)
#         label = label.to(device)
        
#         gen_label = torch.randint(0, n_classes, size=(b_size,))
#         gen_label = gen_label.to(device)
        
#         data_fake = generator(noise, gen_label)
#         loss_g = train_generator(opt_G, data_fake, gen_label)
        
#         for stpe in range(k):
#             data_fake = generator(noise, label).detach()
#             data_real = image 
#             loss_d = train_discriminator(opt_D, data_real, data_fake, label)
        
#         print(f"Loss D: {loss_d}, Loss G: {loss_g}")
    
#     gen_label = torch.randint(0, n_classes, size=(sample_size,))
#     generated_img = generator(latent_vector(sample_size, latent_dim), gen_label).cpu().detach()
#     generated_img = make_grid(generated_img)
#     save_generated_image(generated_img, f"./output/image/gen_img{epoch+1}.png")
    
#     images.append(generated_img)
    
#     epoch_loss_g = loss_g/epoch
#     epoch_loss_d = loss_d/epoch
    
#     losses_g.append(epoch_loss_g)
#     losses_d.append(epoch_loss_d)
    
#     print(f"{epoch+1}/{epochs}")
#     print(f"Generator Loss: {epoch_loss_g:.4f}, Discriminator Loss: {epoch_loss_d:.4f}")
    

In [16]:
FloatTensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor

In [17]:
def sample_image(n_row, batches_done):
    # Create noise
    z = Variable(FloatTensor(np.random.normal(0,1,(n_row**2, latent_dim))))
    
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    
    labels = Variable(LongTensor(labels))
    
    gen_imgs = generator(z, labels)
    
    save_image(gen_imgs.data, f"./output/image/{batches_done}.png", nrow=n_row, normalize=True)

In [ ]:
for epoch in range(epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        b_size = imgs.size(0)
        
        #Adversarial ground truths
        valid = Variable(FloatTensor(b_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(b_size, 1).fill_(0.0), requires_grad=False)
        
        
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))
        
        #Train Generator 
        opt_G.zero_grad()
        z = Variable(FloatTensor(np.random.normal(0, 1, (b_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, b_size)))
        
        gen_imgs = generator(z, gen_labels)
        
        validty = discriminator(gen_imgs, gen_labels)
        g_loss = criterion(validty, valid)
        
        g_loss.backward()
        opt_G.step()
        
        #Train Discriminator
        opt_D.zero_grad()
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = criterion(validity_real, valid)
        
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = criterion(validity_fake, fake)
        
        d_loss = (d_real_loss+d_fake_loss)/2
        
        d_loss.backward()
        opt_D.step()
        
        print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}]\
              [D Loss: {d_loss.item() : .4f}][G Loss: {g_loss.item() : .4f}]")
        
        batches_done = epoch*len(train_loader)+i
        if batches_done%sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

[Epoch 0/200] [Batch 0/469]              [D Loss:  0.5585][G Loss:  1.0326]
[Epoch 0/200] [Batch 1/469]              [D Loss:  0.3224][G Loss:  1.0027]
[Epoch 0/200] [Batch 2/469]              [D Loss:  0.1607][G Loss:  0.9768]
[Epoch 0/200] [Batch 3/469]              [D Loss:  0.0544][G Loss:  0.9540]
[Epoch 0/200] [Batch 4/469]              [D Loss:  0.0128][G Loss:  0.9279]
[Epoch 0/200] [Batch 5/469]              [D Loss:  0.0693][G Loss:  0.9021]
[Epoch 0/200] [Batch 6/469]              [D Loss:  0.1119][G Loss:  0.8899]
[Epoch 0/200] [Batch 7/469]              [D Loss:  0.0738][G Loss:  0.8720]
[Epoch 0/200] [Batch 8/469]              [D Loss:  0.0434][G Loss:  0.8785]
[Epoch 0/200] [Batch 9/469]              [D Loss:  0.0138][G Loss:  0.8702]
[Epoch 0/200] [Batch 10/469]              [D Loss:  0.0170][G Loss:  0.8726]
[Epoch 0/200] [Batch 11/469]              [D Loss:  0.0313][G Loss:  0.8789]
[Epoch 0/200] [Batch 12/469]              [D Loss:  0.0486][G Loss:  0.8726]
[Epoch 0/

[Epoch 0/200] [Batch 108/469]              [D Loss:  0.0123][G Loss:  0.9762]
[Epoch 0/200] [Batch 109/469]              [D Loss:  0.0135][G Loss:  1.0591]
[Epoch 0/200] [Batch 110/469]              [D Loss:  0.0139][G Loss:  0.9313]
[Epoch 0/200] [Batch 111/469]              [D Loss:  0.0157][G Loss:  0.8791]
[Epoch 0/200] [Batch 112/469]              [D Loss:  0.0154][G Loss:  0.9518]
[Epoch 0/200] [Batch 113/469]              [D Loss:  0.0149][G Loss:  1.0103]
[Epoch 0/200] [Batch 114/469]              [D Loss:  0.0156][G Loss:  0.9258]
[Epoch 0/200] [Batch 115/469]              [D Loss:  0.0146][G Loss:  0.9012]
[Epoch 0/200] [Batch 116/469]              [D Loss:  0.0146][G Loss:  0.9196]
[Epoch 0/200] [Batch 117/469]              [D Loss:  0.0149][G Loss:  1.0131]
[Epoch 0/200] [Batch 118/469]              [D Loss:  0.0160][G Loss:  1.0044]
[Epoch 0/200] [Batch 119/469]              [D Loss:  0.0159][G Loss:  0.9292]
[Epoch 0/200] [Batch 120/469]              [D Loss:  0.0184][G L

[Epoch 0/200] [Batch 214/469]              [D Loss:  0.0102][G Loss:  0.9508]
[Epoch 0/200] [Batch 215/469]              [D Loss:  0.0096][G Loss:  0.9553]
[Epoch 0/200] [Batch 216/469]              [D Loss:  0.0096][G Loss:  1.0737]
[Epoch 0/200] [Batch 217/469]              [D Loss:  0.0103][G Loss:  1.0932]
[Epoch 0/200] [Batch 218/469]              [D Loss:  0.0100][G Loss:  1.0087]
[Epoch 0/200] [Batch 219/469]              [D Loss:  0.0114][G Loss:  0.9785]
[Epoch 0/200] [Batch 220/469]              [D Loss:  0.0113][G Loss:  0.9750]
[Epoch 0/200] [Batch 221/469]              [D Loss:  0.0106][G Loss:  0.9657]
[Epoch 0/200] [Batch 222/469]              [D Loss:  0.0108][G Loss:  0.9454]
[Epoch 0/200] [Batch 223/469]              [D Loss:  0.0119][G Loss:  0.9481]
[Epoch 0/200] [Batch 224/469]              [D Loss:  0.0095][G Loss:  1.0194]
[Epoch 0/200] [Batch 225/469]              [D Loss:  0.0099][G Loss:  0.9938]
[Epoch 0/200] [Batch 226/469]              [D Loss:  0.0087][G L

[Epoch 0/200] [Batch 320/469]              [D Loss:  0.0175][G Loss:  0.8915]
[Epoch 0/200] [Batch 321/469]              [D Loss:  0.0151][G Loss:  0.9525]
[Epoch 0/200] [Batch 322/469]              [D Loss:  0.0201][G Loss:  1.0862]
[Epoch 0/200] [Batch 323/469]              [D Loss:  0.0166][G Loss:  0.9144]
[Epoch 0/200] [Batch 324/469]              [D Loss:  0.0140][G Loss:  0.9087]
[Epoch 0/200] [Batch 325/469]              [D Loss:  0.0128][G Loss:  0.9463]
[Epoch 0/200] [Batch 326/469]              [D Loss:  0.0183][G Loss:  0.9135]
[Epoch 0/200] [Batch 327/469]              [D Loss:  0.0183][G Loss:  0.9406]
[Epoch 0/200] [Batch 328/469]              [D Loss:  0.0152][G Loss:  0.9277]
[Epoch 0/200] [Batch 329/469]              [D Loss:  0.0154][G Loss:  1.0610]
[Epoch 0/200] [Batch 330/469]              [D Loss:  0.0131][G Loss:  0.9957]
[Epoch 0/200] [Batch 331/469]              [D Loss:  0.0156][G Loss:  0.9849]
[Epoch 0/200] [Batch 332/469]              [D Loss:  0.0162][G L

[Epoch 0/200] [Batch 426/469]              [D Loss:  0.0137][G Loss:  0.9639]
[Epoch 0/200] [Batch 427/469]              [D Loss:  0.0110][G Loss:  0.9939]
[Epoch 0/200] [Batch 428/469]              [D Loss:  0.0113][G Loss:  0.9218]
[Epoch 0/200] [Batch 429/469]              [D Loss:  0.0144][G Loss:  0.9356]
[Epoch 0/200] [Batch 430/469]              [D Loss:  0.0181][G Loss:  1.1647]
[Epoch 0/200] [Batch 431/469]              [D Loss:  0.0171][G Loss:  0.7722]
[Epoch 0/200] [Batch 432/469]              [D Loss:  0.0135][G Loss:  0.9766]
[Epoch 0/200] [Batch 433/469]              [D Loss:  0.0195][G Loss:  1.1385]
[Epoch 0/200] [Batch 434/469]              [D Loss:  0.0123][G Loss:  0.8674]
[Epoch 0/200] [Batch 435/469]              [D Loss:  0.0234][G Loss:  0.8073]
[Epoch 0/200] [Batch 436/469]              [D Loss:  0.0306][G Loss:  1.2637]
[Epoch 0/200] [Batch 437/469]              [D Loss:  0.0148][G Loss:  0.9646]
[Epoch 0/200] [Batch 438/469]              [D Loss:  0.0336][G L

...

[Epoch 10/200] [Batch 305/469]              [D Loss:  0.0281][G Loss:  0.9238]
[Epoch 10/200] [Batch 306/469]              [D Loss:  0.0592][G Loss:  0.6961]
[Epoch 10/200] [Batch 307/469]              [D Loss:  0.0278][G Loss:  0.8532]
[Epoch 10/200] [Batch 308/469]              [D Loss:  0.0323][G Loss:  1.0422]
[Epoch 10/200] [Batch 309/469]              [D Loss:  0.0450][G Loss:  1.0664]
[Epoch 10/200] [Batch 310/469]              [D Loss:  0.0198][G Loss:  0.9609]
[Epoch 10/200] [Batch 311/469]              [D Loss:  0.0572][G Loss:  0.6880]
[Epoch 10/200] [Batch 312/469]              [D Loss:  0.0262][G Loss:  0.8675]
[Epoch 10/200] [Batch 313/469]              [D Loss:  0.0268][G Loss:  0.9905]
[Epoch 10/200] [Batch 314/469]              [D Loss:  0.0227][G Loss:  1.0108]
[Epoch 10/200] [Batch 315/469]              [D Loss:  0.0272][G Loss:  1.0426]
[Epoch 10/200] [Batch 316/469]              [D Loss:  0.0112][G Loss:  0.9382]
[Epoch 10/200] [Batch 317/469]              [D Loss:

[Epoch 10/200] [Batch 409/469]              [D Loss:  0.0309][G Loss:  0.8249]
[Epoch 10/200] [Batch 410/469]              [D Loss:  0.0492][G Loss:  0.9768]
[Epoch 10/200] [Batch 411/469]              [D Loss:  0.0334][G Loss:  0.9393]
[Epoch 10/200] [Batch 412/469]              [D Loss:  0.0340][G Loss:  0.7720]
[Epoch 10/200] [Batch 413/469]              [D Loss:  0.0264][G Loss:  0.8725]
[Epoch 10/200] [Batch 414/469]              [D Loss:  0.0289][G Loss:  0.9575]
[Epoch 10/200] [Batch 415/469]              [D Loss:  0.0318][G Loss:  0.9649]
[Epoch 10/200] [Batch 416/469]              [D Loss:  0.0207][G Loss:  0.8879]
[Epoch 10/200] [Batch 417/469]              [D Loss:  0.0413][G Loss:  0.7768]
[Epoch 10/200] [Batch 418/469]              [D Loss:  0.0250][G Loss:  0.9828]
[Epoch 10/200] [Batch 419/469]              [D Loss:  0.0254][G Loss:  1.0092]
[Epoch 10/200] [Batch 420/469]              [D Loss:  0.0157][G Loss:  0.9395]
[Epoch 10/200] [Batch 421/469]              [D Loss:

[Epoch 11/200] [Batch 45/469]              [D Loss:  0.0262][G Loss:  1.0143]
[Epoch 11/200] [Batch 46/469]              [D Loss:  0.0321][G Loss:  1.0410]
[Epoch 11/200] [Batch 47/469]              [D Loss:  0.0289][G Loss:  0.9242]
[Epoch 11/200] [Batch 48/469]              [D Loss:  0.0349][G Loss:  0.8224]
[Epoch 11/200] [Batch 49/469]              [D Loss:  0.0294][G Loss:  0.8662]
[Epoch 11/200] [Batch 50/469]              [D Loss:  0.0173][G Loss:  0.9687]
[Epoch 11/200] [Batch 51/469]              [D Loss:  0.0280][G Loss:  0.9548]
[Epoch 11/200] [Batch 52/469]              [D Loss:  0.0273][G Loss:  0.9592]
[Epoch 11/200] [Batch 53/469]              [D Loss:  0.0303][G Loss:  0.8991]
[Epoch 11/200] [Batch 54/469]              [D Loss:  0.0279][G Loss:  0.8895]
[Epoch 11/200] [Batch 55/469]              [D Loss:  0.0175][G Loss:  0.9381]
[Epoch 11/200] [Batch 56/469]              [D Loss:  0.0159][G Loss:  0.9138]
[Epoch 11/200] [Batch 57/469]              [D Loss:  0.0311][G L

[Epoch 11/200] [Batch 150/469]              [D Loss:  0.0352][G Loss:  0.8583]
[Epoch 11/200] [Batch 151/469]              [D Loss:  0.0290][G Loss:  0.8875]
[Epoch 11/200] [Batch 152/469]              [D Loss:  0.0286][G Loss:  0.8700]
[Epoch 11/200] [Batch 153/469]              [D Loss:  0.0332][G Loss:  0.9206]
[Epoch 11/200] [Batch 154/469]              [D Loss:  0.0313][G Loss:  0.8691]
[Epoch 11/200] [Batch 155/469]              [D Loss:  0.0345][G Loss:  0.8441]
[Epoch 11/200] [Batch 156/469]              [D Loss:  0.0258][G Loss:  0.8979]
[Epoch 11/200] [Batch 157/469]              [D Loss:  0.0276][G Loss:  0.9545]
[Epoch 11/200] [Batch 158/469]              [D Loss:  0.0252][G Loss:  0.9534]
[Epoch 11/200] [Batch 159/469]              [D Loss:  0.0294][G Loss:  0.8179]
[Epoch 11/200] [Batch 160/469]              [D Loss:  0.0364][G Loss:  1.0078]
[Epoch 11/200] [Batch 161/469]              [D Loss:  0.0263][G Loss:  0.8961]
[Epoch 11/200] [Batch 162/469]              [D Loss:

[Epoch 11/200] [Batch 255/469]              [D Loss:  0.0303][G Loss:  0.8525]
[Epoch 11/200] [Batch 256/469]              [D Loss:  0.0395][G Loss:  0.9996]
[Epoch 11/200] [Batch 257/469]              [D Loss:  0.0266][G Loss:  0.8761]
[Epoch 11/200] [Batch 258/469]              [D Loss:  0.0348][G Loss:  0.8239]
[Epoch 11/200] [Batch 259/469]              [D Loss:  0.0185][G Loss:  0.9459]
[Epoch 11/200] [Batch 260/469]              [D Loss:  0.0243][G Loss:  0.9546]
[Epoch 11/200] [Batch 261/469]              [D Loss:  0.0280][G Loss:  0.9562]
[Epoch 11/200] [Batch 262/469]              [D Loss:  0.0354][G Loss:  0.8534]
[Epoch 11/200] [Batch 263/469]              [D Loss:  0.0284][G Loss:  0.8846]
[Epoch 11/200] [Batch 264/469]              [D Loss:  0.0356][G Loss:  0.9612]
[Epoch 11/200] [Batch 265/469]              [D Loss:  0.0333][G Loss:  0.9385]
[Epoch 11/200] [Batch 266/469]              [D Loss:  0.0295][G Loss:  0.8399]
[Epoch 11/200] [Batch 267/469]              [D Loss: